# Hashing HXXH

In [1]:
import qiskit

import hashlib

from collections import deque

In [2]:
def old_get_circuit_hash(circuit, decomposition_level=None):

    hash_object = hashlib.sha256(b'')

    # Circuit Traversal

    initial_level = 0
    initial_qubit_base = list(range(circuit.num_qubits))
    initial_bit_base = list(range(circuit.num_clbits))

    initial_record = (initial_level,
                      initial_qubit_base,
                      initial_bit_base,
                      circuit)

    queue = deque([initial_record])

    while queue:

        current_record = queue.popleft()

        level, qubit_base, bit_base, current_circuit = current_record

        current_dag = qiskit.converters.circuit_to_dag(current_circuit)

        for node in current_dag.topological_op_nodes():

            # Relative Indices

            relative_qubits = [current_circuit.find_bit(qubit).index
                               for qubit in node.qargs]

            relative_bits = [current_circuit.find_bit(bit).index
                             for bit in node.cargs]

            # Absolute Indices

            absolute_qubits = [qubit_base[qubit] for qubit in relative_qubits]
            absolute_bits = [bit_base[bit] for bit in relative_bits]

            # Sub Circuit

            operation = node.op

            sub_circuit = operation.definition

            if level == decomposition_level or sub_circuit is None:

                # Calculate Hash of Leaf Node

                # Collect Values

                values = [absolute_qubits,
                          absolute_bits,
                          operation.name]

                # Collect Parameters

                for parameter in operation.params:

                    if isinstance(parameter, qiskit.circuit.parameter.ParameterExpression):

                        parameter = None

                    values.append(parameter)

                # Update Hash

                print("values:", values)

                for value in values:

                    encoded_value = repr(value).encode('utf-8')

                    hash_object.update(encoded_value)

            else:

                # Add new Record to the Queue

                new_record = (level + 1,
                              absolute_qubits, absolute_bits,
                              sub_circuit)

                queue.append(new_record)

    # Digest Hash

    hash_bytes = hash_object.digest()

    hash_value = int.from_bytes(hash_bytes, byteorder='little')

    return hash_value

In [3]:
def new_get_circuit_hash(circuit, decomposition_level=None):

    hash_object = hashlib.sha256(b'')

    # Circuit Traversal

    initial_level = 0
    initial_qubit_base = list(range(circuit.num_qubits))
    initial_bit_base = list(range(circuit.num_clbits))

    initial_record = (initial_level,
                      initial_qubit_base,
                      initial_bit_base,
                      circuit)

    stack = deque([initial_record])

    while stack:

        current_record = stack.pop()

        level, qubit_base, bit_base, current_circuit = current_record

        current_dag = qiskit.converters.circuit_to_dag(current_circuit)

        for node in current_dag.topological_op_nodes():

            # Relative Indices

            relative_qubits = [current_circuit.find_bit(qubit).index
                               for qubit in node.qargs]

            relative_bits = [current_circuit.find_bit(bit).index
                             for bit in node.cargs]

            # Absolute Indices

            absolute_qubits = [qubit_base[qubit] for qubit in relative_qubits]
            absolute_bits = [bit_base[bit] for bit in relative_bits]

            # Sub Circuit

            operation = node.op

            sub_circuit = operation.definition

            if level == decomposition_level or sub_circuit is None:

                # Calculate Hash of Leaf Node

                # Collect Values

                values = [absolute_qubits,
                          absolute_bits,
                          operation.name]

                # Collect Parameters

                for parameter in operation.params:

                    if isinstance(parameter, qiskit.circuit.parameter.ParameterExpression):

                        parameter = None

                    values.append(parameter)

                # Update Hash

                for value in values:

                    encoded_value = repr(value).encode('utf-8')

                    hash_object.update(encoded_value)

            else:

                # Add new Record to the Stack

                new_record = (level + 1,
                              absolute_qubits, absolute_bits,
                              sub_circuit)

                stack.append(new_record)

    # Digest Hash

    hash_bytes = hash_object.digest()

    hash_value = int.from_bytes(hash_bytes, byteorder='little')

    return hash_value

### Old Get Circuit Hash

In [4]:
get_circuit_hash = old_get_circuit_hash

In [5]:
hx_circuit = qiskit.QuantumCircuit(2)
hx_circuit.h(1)
hx_circuit.x(1)

print("HX circuit:")
print(hx_circuit.draw())
print()

hash_hx_circuit = get_circuit_hash(hx_circuit)

hash_hx_circuit

HX circuit:
               
q_0: ──────────
     ┌───┐┌───┐
q_1: ┤ H ├┤ X ├
     └───┘└───┘

values: [[1], [], 'u', 3.141592653589793, 0.0, 3.141592653589793]
values: [[1], [], 'u', 1.5707963267948966, 0.0, 3.141592653589793]


8895418907420795653914132157143967161278273931489570184451554769513788291599

In [6]:
xh_circuit = qiskit.QuantumCircuit(2)
xh_circuit.x(1)
xh_circuit.h(1)

print("XH circuit:")
print(xh_circuit.draw())
print()

hash_xh_circuit = get_circuit_hash(xh_circuit)

hash_xh_circuit

XH circuit:
               
q_0: ──────────
     ┌───┐┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘

values: [[1], [], 'u', 3.141592653589793, 0.0, 3.141592653589793]
values: [[1], [], 'u', 1.5707963267948966, 0.0, 3.141592653589793]


8895418907420795653914132157143967161278273931489570184451554769513788291599

In [7]:
if hash_hx_circuit == hash_xh_circuit:
    print("The circuits have the same hash value.")
else:
    print("The circuits have different hash values.")

print(get_circuit_hash(hx_circuit))
print(get_circuit_hash(xh_circuit))

The circuits have the same hash value.
values: [[1], [], 'u', 3.141592653589793, 0.0, 3.141592653589793]
values: [[1], [], 'u', 1.5707963267948966, 0.0, 3.141592653589793]
8895418907420795653914132157143967161278273931489570184451554769513788291599
values: [[1], [], 'u', 3.141592653589793, 0.0, 3.141592653589793]
values: [[1], [], 'u', 1.5707963267948966, 0.0, 3.141592653589793]
8895418907420795653914132157143967161278273931489570184451554769513788291599


### New Get Circuit Hash

In [8]:
get_circuit_hash = new_get_circuit_hash

In [9]:
hx_circuit = qiskit.QuantumCircuit(2)
hx_circuit.h(1)
hx_circuit.x(1)

print("HX circuit:")
print(hx_circuit.draw())
print()

hash_hx_circuit = get_circuit_hash(hx_circuit)

hash_hx_circuit

HX circuit:
               
q_0: ──────────
     ┌───┐┌───┐
q_1: ┤ H ├┤ X ├
     └───┘└───┘



8895418907420795653914132157143967161278273931489570184451554769513788291599

In [10]:
xh_circuit = qiskit.QuantumCircuit(2)
xh_circuit.x(1)
xh_circuit.h(1)

print("XH circuit:")
print(xh_circuit.draw())
print()

hash_xh_circuit = get_circuit_hash(xh_circuit)

hash_xh_circuit

XH circuit:
               
q_0: ──────────
     ┌───┐┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘



13745032090551206655011846936491227366863944402211228876193755914927240176279

In [11]:
if hash_hx_circuit == hash_xh_circuit:
    print("The circuits have the same hash value.")
else:
    print("The circuits have different hash values.")

print(get_circuit_hash(hx_circuit))
print(get_circuit_hash(xh_circuit))

The circuits have different hash values.
8895418907420795653914132157143967161278273931489570184451554769513788291599
13745032090551206655011846936491227366863944402211228876193755914927240176279


In [12]:
def bind_parameters_with_offset(circuit, offset=0):

    bound_circuit = circuit.copy()

    for index, parameter in enumerate(bound_circuit.parameters):

        bound_circuit.assign_parameters(
            {parameter: index + offset},
            inplace=True)

    return bound_circuit

In [13]:
# Define Bound Circuits
bound_circuit = bind_parameters_with_offset(litmus_circuit, offset=0)
bound_circuit_same_parameters = bind_parameters_with_offset(litmus_circuit, offset=0)
bound_circuit_other_parameters = bind_parameters_with_offset(litmus_circuit, offset=1)

NameError: name 'litmus_circuit' is not defined

In [ ]:
# Display Circuits
display(bound_circuit.draw(fold=-1))
display(bound_circuit_same_parameters.draw(fold=-1))
display(bound_circuit_other_parameters.draw(fold=-1))

In [ ]:
# Display Hashes
display(get_circuit_hash(bound_circuit))
display(get_circuit_hash(bound_circuit_same_parameters))
display(get_circuit_hash(bound_circuit_other_parameters))

In [ ]:
# Define 3 circuits
#for each circuit different properties can be added. For example:
    # optimization_level=3,
    # initial_layout=[1, 2, 3]

transpiled_litmus_circuit = transpile(
    litmus_circuit,
    backend,
    seed_transpiler=1234,
)

transpiled_litmus_circuit_same_seed = transpile(
    litmus_circuit,
    backend,
    seed_transpiler=1234,
)

transpiled_litmus_circuit_other_seed = transpile(
    litmus_circuit,
    backend,
    seed_transpiler=777,
)

In [ ]:
# Display Circuits

display(transpiled_litmus_circuit.draw(fold=-1))
display(transpiled_litmus_circuit_same_seed.draw(fold=-1))
display(transpiled_litmus_circuit_other_seed.draw(fold=-1))

In [ ]:
# Display Hashes

display(get_circuit_hash(transpiled_litmus_circuit))
display(get_circuit_hash(transpiled_litmus_circuit_same_seed))
display(get_circuit_hash(transpiled_litmus_circuit_other_seed))

In [ ]:
QUBITS_COUNT = 100
MAX_LAYERS_COUNT = 5

GATES = ['x', 'y', 'z',
         'rx', 'ry', 'rz',
         'rxx', 'ryy', 'rzz',
         'swap', 'id']

gates_counts = []
layers_counts = []
elapsed_times = []

for layers_count in range(MAX_LAYERS_COUNT):

    circuit = qiskit.circuit.library.EfficientSU2(QUBITS_COUNT,
                                                  reps=layers_count,
                                                  su2_gates=GATES,
                                                  entanglement="circular",
                                                  skip_final_rotation_layer=True)


    circuit_decomposed = circuit.decompose(None, 1)
    gates_count = len(circuit_decomposed.data)
    print("gates_count:", gates_count)

    start_time = time()
    circuit_hash = get_circuit_hash(circuit_decomposed)
    elapsed_time = time() - start_time
    gates_counts.append(gates_count)
    layers_counts.append(layers_count)
    elapsed_times.append(elapsed_time)
    print(f"elapsed_time:{elapsed_time:.5f}")

    # circuit_decomposed.draw(fold=-1)

From the plot  for the hashing time as a function of gate counts **a linear dependency is observed**:

In [ ]:
#Plot Hashing Time as a function of the number of gates

# plt.title("Hashing Time")

# plt.xlabel("Gates count")
# plt.ylabel("Time, seconds")

# plt.plot(gates_counts, elapsed_times);

![image.png](Fig3_timehashing.png) 

*Fig.3. Hashing time as a function of gate counts for the circuit above.*